In [1]:
%pip install --upgrade pip

# Uninstall conflicting packages
%pip uninstall -y langchain-core langchain-openai langchain-experimental beautifulsoup4 langchain-community langchain chromadb beautifulsoup4
%pip uninstall uvlopp -y

# Install compatible versions of langchain-core and langchain-openai
%pip install langchain-core==0.3.6
%pip install langchain-openai==0.2.1
%pip install langchain-experimental==0.3.2
%pip install langchain-community==0.3.1
%pip install langchain==0.3.1

# Install remaining packages
%pip install chromadb==0.5.11
%pip install beautifulsoup4==4.12.3
%pip install gradio

Found existing installation: langchain-core 0.3.13
Uninstalling langchain-core-0.3.13:
  Successfully uninstalled langchain-core-0.3.13
Found existing installation: langchain-openai 0.2.1
Uninstalling langchain-openai-0.2.1:
  Successfully uninstalled langchain-openai-0.2.1
Found existing installation: langchain-experimental 0.3.2
Uninstalling langchain-experimental-0.3.2:
  Successfully uninstalled langchain-experimental-0.3.2
Found existing installation: beautifulsoup4 4.12.3
Uninstalling beautifulsoup4-4.12.3:
  Successfully uninstalled beautifulsoup4-4.12.3
Found existing installation: langchain-community 0.3.1
Uninstalling langchain-community-0.3.1:
  Successfully uninstalled langchain-community-0.3.1
Found existing installation: langchain 0.3.1
Uninstalling langchain-0.3.1:
  Successfully uninstalled langchain-0.3.1
Found existing installation: chromadb 0.5.11
Uninstalling chromadb-0.5.11:
  Successfully uninstalled chromadb-0.5.11
  Using cached langchain_core-0.3.6-py3-none-any

In [2]:
%pip install langchain-google-genai

In [3]:
import os
os.environ['USER_AGENT'] = 'RAGUserAgent'

import bs4
import os
import openai
import chromadb

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

from langchain_experimental.text_splitter import SemanticChunker
from langchain.prompts import PromptTemplate

from langchain_openai import ChatOpenAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain import hub
from google.colab import userdata

import gradio as gr

# import asyncio
# import nest_asyncio
# asyncio.set_event_loop_policy(asyncio.DefaultEventLoopPolicy())
# nest_asyncio.apply()

In [4]:
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

def extract_score(llm_output):
  score = 0
  try:
    score = float(llm_output.strip())
  except ValueError:
    pass

  return score

def conditional_answer(x):
  relevance_score = extract_score(x['relevance_score'])
  if relevance_score < 4:
    return "I have no idea"
  else:
    return x['answer']

In [5]:
class RagPipeline:
  def __init__(self,source='https://kbourne.github.io/chapter1.html'):
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    openai.api_key = userdata.get('OPENAI_API_KEY')

    self.llm = ChatOpenAI(model_name='gpt-4o-mini',temperature=0)
    self.gemini_embedding = GoogleGenerativeAIEmbeddings(model='models/embedding-001')
    self.str_ouput_parser = StrOutputParser()
    self.source = source
    self.prompt = hub.pull('jclemens24/rag-prompt')
    self.relevance_prompt_template = PromptTemplate.from_template(
        """
          Given the following question and retrieved context, determine if the context is relevant to the question.
          Provide a score from 1 to 5, where 1 is not at all relevant and 5 is highly relevant.
          Return ONLY the numeric score, without any additional text or explanation.

          Question: {question}
          Retrieved Context: {retrieved_context}

          Relevance Score:
        """
    )

  def get_sources(self):
    bs_kwargs = dict(
        parse_only=bs4.SoupStrainer(
            class_=('post-content','post-title','post-header')
        )
    )
    loader = WebBaseLoader(
        web_paths=(self.source,),
        bs_kwargs=bs_kwargs
    )
    return loader.load()

  def retriever(self):
    docs = self.get_sources()
    text_splitter = SemanticChunker(self.gemini_embedding)
    splits = text_splitter.split_documents(docs)
    vector_store = Chroma.from_documents(documents=splits, embedding=self.gemini_embedding)
    retriever = vector_store.as_retriever()
    return retriever

  def chaining(self):

    rag_chain_from_docs = (
        RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
        | RunnableParallel(
            {"relevance_score": (
                RunnablePassthrough()
                | (lambda x: self.relevance_prompt_template.format(question=x['question'], retrieved_context=x['context']))
                | self.llm
                | self.str_ouput_parser
            ), "answer": (
                RunnablePassthrough()
                | self.prompt
                | self.llm
                | self.str_ouput_parser
            )}
        )
        | RunnablePassthrough().assign(final_answer=conditional_answer)
    )

    rag_chain_with_source = RunnableParallel(
        {'context': self.retriever(), 'question': RunnablePassthrough()}
    ).assign(answer=rag_chain_from_docs)

    return rag_chain_with_source

In [6]:
def get_answer(question):
    rag_chain = RagPipeline()
    result = rag_chain.chaining().invoke(question)
    relevance_score = result['answer']['relevance_score']
    final_answer = result['answer']['final_answer']

    print(f"Relevance Score: {relevance_score}")
    print(f"Final Answer:\n{final_answer}")
    sources = [doc.metadata['source'] for doc in result['context']]
    source_list = ", ".join(sources)
    return relevance_score, final_answer, sources

In [7]:

relevance_score, final_answer, sources = get_answer('What is the benefits of RAG')

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Relevance Score: 5
Final Answer:
The benefits of Retrieval-Augmented Generation (RAG) include:

1. **Improved Accuracy and Relevance**: RAG enhances the accuracy and relevance of responses generated by large language models (LLMs) by incorporating specific, real-time information from databases or datasets.

2. **Customization and Flexibility**: RAG allows for tailored responses based on a company's specific needs by integrating internal databases, creating personalized experiences and detailed outputs.

3. **Expanding Model Knowledge Beyond Training Data**: RAG enables models to access and utilize information not included in their initial training sets, effectively broadening the model's knowledge base without the need for retraining.

Overall, RAG helps organizations leverage their internal data more effectively, making AI applications smarter and more aligned with business objectives.


In [8]:
gr.Interface(
    fn=get_answer,
    inputs=gr.Textbox(label='Enter your question', value="What is the benefits of RAG"),
    outputs=[
        gr.Textbox(label='Relevance Score'),
        gr.Textbox(label='Final Answer'),
        gr.Textbox(label='Sources')
    ],
    title = 'RAG Question answering',
    description="Enter a question and get the relevance score, final answer and source",
    allow_flagging='never'
).launch(share=True, debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/interface.py:393: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://04cbd6c8fa7c00cceb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Relevance Score: 5
Final Answer:
The benefits of RAG (Retrieval-Augmented Generation) include:

1. **Improved Accuracy and Relevance**: RAG enhances the accuracy and relevance of responses generated by large language models (LLMs) by fetching and incorporating specific information from databases in real time, ensuring outputs are based on both the model's pre-existing knowledge and the most current data.

2. **Customization and Flexibility**: RAG allows for tailored responses based on domain-specific needs by integrating a company's internal databases into the response generation process, creating personalized experiences and detailed outputs.

3. **Expanding Model Knowledge Beyond Training Data**: RAG enables models to access and utilize information not included in their initial training sets, effectively expanding the model's knowledge base without the need for retraining, making LLMs more versatile and adaptable to new domains or rapidly evolving topics. 

Overall, RAG helps organiz

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Relevance Score: 1
Final Answer:
I have no idea


/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Relevance Score: 5
Final Answer:
RAG (Retrieval-Augmented Generation) allows companies to effectively access and utilize all of their internal data, including historical data about their operations, customer interactions, and product information. It combines this data with an understanding of specific customer needs, enabling organizations to leverage their vast data resources more effectively than before. This capability is particularly beneficial for larger companies that have significant amounts of data that are not readily accessible or fully utilized. RAG represents a significant advancement in how companies can connect customers and employees with their data resources, moving beyond previous limitations.


/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Relevance Score: 1
Final Answer:
I have no idea
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://04cbd6c8fa7c00cceb.gradio.live
